In [ ]:
#Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import spectrogram
from glob import glob


FS = 12000
WIN_SIZE = FS  # 1 segundo
STEP = FS

# Caminhos
DATA_DIR = "../data"
OUTPUT_DIR = "../images"

DATASETS = [
    {"name": "cwru", "raw_dir": "raw_cwru", "img_dir": "cwru"},
    {"name": "hust", "raw_dir": "raw_hust", "img_dir": "hust"},
    #{"name": "ottawa", "raw_dir": "raw_ottawa", "img_dir": "ottawa"},
]

# Criação das classes se ainda não existirem
CLASSES = ["normal", "inner", "outer", "ball"]

# Garante que todas pastas de saída por classe existem
for ds in DATASETS:
    for cls in CLASSES:
        os.makedirs(os.path.join(OUTPUT_DIR, ds["img_dir"], cls), exist_ok=True)

# Mapeamento de classes baseado no número do arquivo
def get_fault_class(file_name, dataset_name):
    file_name = os.path.basename(file_name).lower()
    if dataset_name == "cwru":
        number = int(''.join(filter(str.isdigit, file_name)))
        if number in [97, 98, 99, 135, 136, 137, 138]:
            return "outer"
        elif number == 100:
            return "ball"
        elif number in [122, 123, 124]:
            return "inner"
        elif number in [109, 110, 111, 112]:
            return "normal"
        
    elif dataset_name == "hust":
        if len(file_name) < 2:
            return None
        fault_type = file_name[0]
        if fault_type == "b":
            return "ball"
        elif fault_type == "i":
            return "inner"
        elif fault_type == "o":
            return "outer"
        elif fault_type == "n":
            return "normal"

        return None




In [7]:
#Função para gerar e salvar espectrogramas
def generate_spectrograms(file_path, dataset_name):
    file_name = os.path.basename(file_path)
    fault_class = get_fault_class(file_name, dataset_name)
    if not fault_class:
        print(f"Arquivo {file_name} ignorado (classe desconhecida).")
        return

    print(f"Processando {file_name} como {fault_class}...")
    
    # Carrega e extrai sinal
    mat = loadmat(file_path)
    if dataset_name == "cwru":
        key = next(k for k in mat.keys() if "_DE_time" in k or "DE_time" in k)
    elif dataset_name == "hust":
        key = next((k for k in mat.keys() if not k.startswith("__") and isinstance(mat[k], np.ndarray)), None)
    signal = mat[key].squeeze()
    signal = signal - np.mean(signal)

    n_windows = (len(signal) - WIN_SIZE) // STEP

    for i in range(n_windows):
        seg = signal[i*STEP : i*STEP + WIN_SIZE]
        f, t, Sxx = spectrogram(seg, fs=FS, nperseg=256, noverlap=128)
        Sxx_log = 10 * np.log10(Sxx + 1e-10)

        plt.figure(figsize=(2.24, 2.24), dpi=100)
        plt.axis('off')
        plt.pcolormesh(t, f, Sxx_log, shading='gouraud', cmap='viridis')
        plt.tight_layout(pad=0)

        out_dir = os.path.join(OUTPUT_DIR, dataset_name, fault_class)
        os.makedirs(out_dir, exist_ok=True)
        out_path = os.path.join(out_dir, f"spec_{os.path.splitext(file_name)[0]}_{i:04d}.png")
        plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
        plt.close()


In [8]:

print("\n📁 Processando CWRU...")
cwru_files = glob(os.path.join(DATA_DIR, "raw_cwru", "*.mat"))
for f in cwru_files:
    generate_spectrograms(f, "cwru")

print("\n📁 Processando HUST...")
hust_files = glob(os.path.join(DATA_DIR, "raw_hust", "*.mat"))
for f in hust_files:
     generate_spectrograms(f, "hust")




📁 Processando CWRU...
Processando 100.mat como ball...
Processando 109.mat como normal...
Processando 110.mat como normal...
Processando 111.mat como normal...
Processando 112.mat como normal...
Processando 122.mat como inner...
Processando 123.mat como inner...
Processando 124.mat como inner...
Arquivo 125.mat ignorado (classe desconhecida).
Processando 135.mat como outer...
Processando 136.mat como outer...
Processando 137.mat como outer...
Processando 138.mat como outer...
Processando 97.mat como outer...
Processando 98.mat como outer...
Processando 99.mat como outer...

📁 Processando HUST...
Processando B500.mat como ball...
Processando B502.mat como ball...
Processando B504.mat como ball...
Processando B600.mat como ball...
Processando B602.mat como ball...
Processando B604.mat como ball...
Processando B700.mat como ball...
Processando B702.mat como ball...
Processando B704.mat como ball...
Processando B800.mat como ball...
Processando B802.mat como ball...
Processando B804.mat c